# Aufgabe
# [Steel Plates Faults Data Set](http://archive.ics.uci.edu/ml/datasets/Steel+Plates+Faults)

 ![](https://img.shields.io/badge/Sector-Stahl-gray.svg)
 ![](https://img.shields.io/badge/Gelabled-Ja-blue.svg)
 ![](<https://img.shields.io/badge/Simulation-Nein-red.svg>)
 ![](https://img.shields.io/badge/Zeireihe-Ja-red.svg)

Parameter | Wert
---- | ----
Datensatz Charakteristiken | Multivariate
Merkmal Charakteristiken	| Integer, Real
Zugehörige Aufgaben	| Klassifikation
Anzahl von Datenpunkten	| 1941
Anhzahl von Merkmalen	| 27
Erstellungsdatum | 2010-10-26
Quelle | UCI Machine Learning Repository
Datensatzgröße | 299KB

## Quelle

Semeion, Research Center of Sciences of Communication, Via Sersale 117, 00128, Rome, Italy.\
www.semeion.it


## Informationen zum Datensatz
Art der abhängigen Variablen (7 Arten von Stahlplattenfehlern):
1. Pastry
2. Z_Scratch (Z Kratzer)
3. K_Scatch (K Kratzer)
4. Stains (Flecken)
5. Dirtiness (Verschmutzung)
6. Bumps (Unebenheiten)
7. Other_Faults (Andere Fehler )


## Informationen zu den Merkmalen

27 unabhängige Merkmale:
1. X_Minimum
2. X_Maximum
3. Y_Minimum
4. Y_Maximum
5. Pixels_Areas
6. X_Perimeter
7. Y_Perimeter
8. Sum_of_Luminosity
9. Minimum_of_Luminosity
10. Maximum_of_Luminosity
11. Length_of_Conveyer
12. TypeOfSteel_A300
13. TypeOfSteel_A400
14. Steel_Plate_Thickness
15. Edges_Index
16. Empty_Index
17. Square_Index
18. Outside_X_Index
19. Edges_X_Index
20. Edges_Y_Index
21. Outside_Global_Index
22. LogOfAreas
23. Log_X_Index
24. Log_Y_Index
25. Orientation_Index
26. Luminosity_Index
27. SigmoidOfAreas


## Referenzen   
- [UCI](http://archive.ics.uci.edu/ml/datasets/Steel+Plates+Faults): Database source link.

## Zitate

1. M Buscema, S Terzi, W Tastle, A New Meta-Classifier,in NAFIPS 2010, Toronto (CANADA),26-28 July 2010, 978-1-4244-7858-6/10 2010 IEEE
2. M Buscema, MetaNet: The Theory of Independent Judges, in Substance Use & Misuse, 33(2), 439-461,1998

In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_absolute_error, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

/scratch/slurm_tmpdir/job_23336263/ipykernel_142419/4009618101.py:9: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
df = pd.read_csv('./data/steel+plates+faults/Faults.NNA', sep = '\t', header = None)
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
1936,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1
1940,1261,1281,87951,87967,103,26,22,11682,101,133,...,-0.2000,-0.1139,0.5296,0,0,0,0,0,0,1


In [3]:
# onehot -> label
label = df.iloc[:, -7:].apply(lambda x: (x * [1,2,3,4,5,6,7]).sum(), axis = 1)
df = df.iloc[:, :-7]
df['label'] = label
df.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,label
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,1
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,1
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,1
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,1
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,1


Den detaillierten Modellierungsprozess finden Sie im Anwendungsfall 1, hier gehen wir direkt zur Modellierung über.

In [5]:
X=df.drop('label', axis=1)
y=df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train.shape, X_test.shape, y_train.shape,y_test.shape

((1358, 27), (583, 27), (1358,), (583,))

In [6]:
sc_x = StandardScaler()
X_train = pd.DataFrame(sc_x.fit_transform(X_train), columns=X.columns.values)
X_test = pd.DataFrame(sc_x.transform(X_test), columns=X.columns.values)

In [7]:
from sklearn.metrics import classification_report

In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

print(classification_report(y_test, model.predict(X_test)))


              precision    recall  f1-score   support

           1       0.64      0.35      0.46        51
           2       0.91      0.80      0.85        61
           3       0.98      0.95      0.97       122
           4       0.88      0.96      0.92        23
           5       1.00      0.85      0.92        13
           6       0.65      0.64      0.65       120
           7       0.65      0.77      0.71       193

    accuracy                           0.76       583
   macro avg       0.82      0.76      0.78       583
weighted avg       0.76      0.76      0.75       583



Die obige Abbildung zeigt die Leistung der maschinellen Lernmethode Random Forest bei dieser Aufgabe. Die Genauigkeit gibt an, wie viele der vom Modell getroffenen Vorhersagen richtig sind, und der Recall gibt den Anteil der entsprechenden Klassen an, die das Modell herausfinden kann. Die Genauigkeit der Kategorie 5 beträgt beispielsweise 1, was bedeutet, dass alle vom Modell Vorhersagen der Kategorie 5 richtig sind. Da das Modell jedoch einige der Datenpunkte, die ursprünglich zur Kategorie 5 gehörten, als andere Kategorien bewertete, führte dies dazu, dass das Modell nur 85 % aller Datenpunkte gefunden hat, die zur Kategorie 5 gehörten. Auch hier zeigt sich, dass ein einzelnes Ergebnis kein guter Indikator für die Leistung des Modells ist.
Der F1-Score ist ein Wert, der aus der Kombination von Recall und Genauigkeit berechnet wird. Größere Werte entsprechen besseren Ergebnissen.

Ist dieses Ergebnis also das optimale Ergebnis, das mit Methoden des maschinellen Lernens erzielt werden kann? Nein. Es gibt zwei verschiedene Möglichkeiten, die Leistung des Modells weiter zu verbessern. Die eine ist das Feature-Engineering, eine Methode zur automatischen Generierung neuer, nützlicher Merkmale, um bessere Ergebnisse mit dem Modell zu erzielen. Die andere ist das automatische maschinelle Lernen, das die Leistung des Modells durch die automatische Auswahl der optimalen Modellparameter verbessert. Im Folgenden wird der erste Ansatz anhand eines Beispiels erläutert.

# Einführung in das Automatische Feature Engineering

In [12]:
tmp = copy.deepcopy(X_train)
tmp['label'] = y_train
tmp.to_csv('./tmp/fault_detection.csv')

In [13]:
import sys
sys.path.append('./src/')
import logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
#logger.setLevel(logging.INFO)
logger.setLevel(logging.WARNING)

In [14]:
from joblib import Parallel, delayed, parallel_backend
import pickle
import torch

# ignore warning
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 50

# MCTS
from mcafe.mct import MC_node, MC_edge, MCFE_tree
from mcafe.state import State
from mcafe.agent import Agent
from mcafe.game import FE
from mcafe.value_network import Value_network
from mcafe.nn_scorer import NN_scorer

Hier wird ein neuronales Netzwerkmodell namens 'Value_network' wird initialisiert:
- Dieses Modell hat eine Eingangsgröße von 26, 
- Eine versteckte Größe von 32
- Eine Batch-First-Konfiguration. Wenn 'batch_first' auf 'True gesetzt' ist, bedeutet dies, dass die Eingabedaten in der Form '(batch_size, sequence_length, input_size)' vorliegen. Das heißt, die erste Dimension des Eingabetensors ist die Batch-Dimension, gefolgt von der Sequenzlänge und schließlich den Merkmalsdimensionen. Wenn 'batch_first' auf 'False' gesetzt ist, wird die Eingabe in der Form '(sequence_length, batch_size, input_size)' erwartet.
- Und ist bidirektional. Wenn 'bidirectional' auf 'True' gesetzt ist, wird das neuronale Netzwerk als bidirektionales Netzwerk konfiguriert. Das bedeutet, dass es Informationen sowohl von vorwärts als auch von rückwärts durch die Sequenz verarbeitet.Die Netzwerkarchitektur hat dann doppelt so viele versteckte Einheiten wie angegeben, da es separate versteckte Einheiten für die vorwärts- und rückwärtsgerichteten Durchgänge gibt.
- Ein logger wird ebenfalls übergeben. Der Logger kann verwendet werden, um Informationen wie Verlustwerte, Metriken, Fortschritt und Fehlermeldungen aufzuzeichnen. Es ist besonders nützlich, um den Trainingsfortschritt zu überwachen, Fehler zu debuggen und die Leistung des Modells zu bewerten.

Danach wird ein Objekt 'NN_scorer' erstellt, das das zuvor initialisierte neuronale Netzwerkmodell verwendet. Der 'NN_scorer' wird verwendet, um das neuronale Netzwerk zu trainieren und zu bewerten. Dabei werden die Trainingsdaten 20 mal in das Neueronale Netz gegeben (daher 20 Epochen).

Dann wird ein Objekt 'FE' (Feature Engineering) erstellt. Es verwendet die CSV-Datei, die zuvor definiert wurde ('./tmp/fault_detection.csv'). Es verwendet die folgenden Parameter:
- Der Parameter art = 'C' zeigt an, dass es sich um ein Klassifikationsproblem handelt. 
- Es wird angegeben, dass das Feature Engineering nicht parallel erfolgen soll (parallel = False). 
- Der Parameter datatype = 'normal' zeigt, dass die Daten im normalen Format vorliegen, d. h. es handelt sich um typische numerische oder kategorische Daten, die für das Feature Engineering verwendet werden können.
- Darüber hinaus wird Subsampling aktiviert. Subsampling bezieht sich auf die Technik, bei der eine Teilmenge der Daten zufällig ausgewählt wird, um das Trainingsset zu reduzieren. Dies kann nützlich sein, wenn die Datenmenge groß ist und das Training lange dauert oder wenn Speicher- oder Rechenressourcen begrenzt sind. Durch Subsampling kann die Trainingszeit verkürzt werden, ohne die Gesamtleistung des Modells stark zu beeinträchtigen, insbesondere wenn die Daten gut verteilt sind.
- Das ausbalancierte Sampling ist deaktiviert. Ausbalanciertes Sampling bezieht sich auf die Technik, bei der die Trainingsdaten so ausgewählt werden, dass jede Klasse in den Daten ungefähr die gleiche Anzahl von Beispielen hat. Dies ist wichtig, um das Modell vor einer Verzerrung zu schützen, wenn eine Klasse im Vergleich zu anderen Klassen überrepräsentiert ist. Durch das Deaktivieren des ausbalancierten Samplings können alle verfügbaren Daten für das Training verwendet werden, unabhängig von der Verteilung der Klassen. Dies kann jedoch dazu führen, dass das Modell für seltene Klassen weniger genau ist oder dass das Modell eine Verzerrung aufweist, wenn die Daten unausgeglichen sind.

Anschließend wird ein Agent-Objekt erstellt, das das vorher erstellte Feature-Engineering-Objekt und den Scorern verwendet. Der Agent hat eine maximale Tiefe von 6. Der Algorithmus hat also eine rekursions Tiefe von 6 und stoppt dann. Der Agent wird ausgeführt und es werden verschiedene Ergebnisse ausgegeben. Die Ergebnisse werden in einer .pkl-Datei gespeichert.

In [15]:
fn = './tmp/fault_detection.csv'

model = Value_network(input_size = 26, 
                       hidden_size = 32, 
                       batch_first = True, 
                       bidirectional = True, 
                       logger = logger)
scorer = NN_scorer(model, num_epochs = 20, logger= logger)
#scorer = NN_scorer(model, num_epochs = 20, logger= logger)
fe = FE(fn, art = 'C', logger = logger, parallel = False, datatype = 'normal', subsampling=True, balanced_sampling=False) # sep = '\t', index_col = 0,
ag = Agent(fe, scorer = scorer, max_depth = 6, logger = logger)
root, init_perf, init_dat, top_five, top_features, buffer= ag.run(path = '../results/')
record_result = {'file': fn, 'game': fe, 'agent': ag, 'top_five': top_five, 'buffer':buffer}
#dict_out.append(record_result)
with open(root + '/results/'+fn.split('/')[-1][:-4]+'.pkl', 'wb') as handle:
    pickle.dump(record_result, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(fe.record_top_five)


2024-04-08 15:11:14,923 - __main__ - WARNING - Original shape of the dataset is (1358, 29)
2024-04-08 15:11:14,930 - __main__ - WARNING - Shape of the dataset after label encoding is (1358, 29)


   Unnamed: 0         0         1         2         3         4         5  \
0           0  0.937875  0.927113 -0.437239 -0.437257 -0.304946 -0.236440   
1           1 -0.762507 -0.852754 -0.728908 -0.728927 -0.316605 -0.221595   
2           2 -0.477169 -0.580802 -0.751391 -0.751413 -0.333739 -0.313636   
3           3 -1.020670 -0.804046  0.305508  0.305511  0.724909  0.428634   
4           4  0.957286  0.945378  0.984134  0.984101 -0.314662 -0.248316   

          6         7         8         9        10        11        12  \
0 -0.124827 -0.339143  1.045784  0.238859 -0.569012 -0.797587  0.797587   
1 -0.128799 -0.355707  0.643932  0.185156 -0.721417  1.253782 -1.253782   
2 -0.162558 -0.376720  1.107607  0.238859 -0.693707 -0.797587  0.797587   
3  0.073755  0.772981 -1.427149 -0.244475 -0.693707 -0.797587  0.797587   
4 -0.130785 -0.359005 -0.128859 -1.211143 -0.652142  1.253782 -1.253782   

         13        14        15        16        17        18        19  \
0 -0.704814

2024-04-08 15:11:18,938 - __main__ - WARNING - The Init mean performance of the given dataset is:	 0.539015637073446, std is:	 0.017781412601804453
2024-04-08 15:11:18,942 - __main__ - WARNING - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2024-04-08 15:11:18,943 - __main__ - WARNING - Round:	0
2024-04-08 15:11:18,943 - __main__ - WARNING - XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2024-04-08 15:11:18,957 - __main__ - WARNING - Run feature transformation: ['prod', 'xgbRegressorTransform', 'radian']


Init performance: 0.539015637073446


2024-04-08 15:11:21,299 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:11:21,300 - __main__ - WARNING - Run feature transformation: ['prod', 'xgbRegressorTransform', 'linearRegressorTransform']
2024-04-08 15:11:21,646 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:11:21,647 - __main__ - WARNING - Run feature transformation: ['prod', 'tanh', 'kernelapproxrbf']
2024-04-08 15:11:24,361 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:11:24,362 - __main__ - WARNING - Run feature transformation: ['prod', 'relu', 'mlpRegressorTransform']
2024-04-08 15:11:30,042 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:11:30,044 - __main__ - WARNING - Run feature transformation: ['prod', 'kernelapproxrbf', 'svrTransform']
2024-04-08 15:11:32,321 - __main__ - WARNING - Add the best feature in set to the best features reco

...

2024-04-08 15:54:49,756 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:54:49,758 - __main__ - WARNING - Run feature transformation: ['zscore', 'ln', 'div', 'mlpRegressorTransform', 'zscore']
2024-04-08 15:54:53,485 - __main__ - WARNING - Run feature transformation: ['zscore', 'ln', 'div', 'mlpRegressorTransform', 'radian']
2024-04-08 15:54:54,301 - __main__ - WARNING - Run feature transformation: ['zscore', 'ln', 'div', 'decisionTreeRegressorTransform', 'minus']
2024-04-08 15:54:55,358 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:54:55,359 - __main__ - WARNING - Run feature transformation: ['zscore', 'ln', 'div', 'prod', 'div']
2024-04-08 15:54:59,617 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:54:59,618 - __main__ - WARNING - Run feature transformation: ['zscore', 'ln', 'div', 'cos', 'mlpRegressorTransform']
2024-04-08 15:55:03,840 - __main_

['zscore', 'add', 'reciprocal', 'negative', 'abs', 'linearRegressorTransform', 'mlpRegressorTransform', 'radian', 'prod', 'timeagg', 'square', 'minus', 'decisionTreeRegressorTransform', 'degree', 'adde', 'cos', 'tanh', 'exp', 'div', 'relu', 'sigmoid', 'kernelapproxrbf', 'xgbRegressorTransform', 'diff', 'ln', 'svrTransform', 'parameter']
W: [1, 1, 1, 3.0, 4, 1, 1, 1, 9.0, 3.0, 3.0, 3.0, 1, 1, 1, 4, 1, 1, 4.0, 2.0, 1, 1, 1, 2.0, 2.0, 3.0]
N: [4, 3, 3, 4, 4, 3, 3, 3, 7, 4, 4, 4, 3, 3, 3, 4, 3, 3, 5, 4, 3, 3, 3, 4, 4, 5]
L: [5.0, 4.0, 4.0, 4.0, 6.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 7.0, 8.0, 8.0, 4.0, 4.0, 9.0, 9.0, 10, 6.0, 8.0, 6.0]
W_rate: [0.16666666666666666, 0.2, 0.2, 0.42857142857142855, 0.4, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.75, 0.375, 0.375, 0.375, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.36363636363636365, 0.1111111111111111, 0.1111111111111111, 0.5, 0.3333333333333333, 0.1, 0.1, 0.09090909090909091, 0.25, 0.

2024-04-08 15:55:20,226 - __main__ - WARNING - Run feature transformation: ['add', 'exp', 'div', 'prod', 'div']
2024-04-08 15:55:32,789 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:55:32,791 - __main__ - WARNING - Run feature transformation: ['add', 'exp', 'div', 'cos', 'mlpRegressorTransform']
2024-04-08 15:55:40,729 - __main__ - WARNING - Run feature transformation: ['add', 'exp', 'div', 'minus', 'svrTransform']
2024-04-08 15:55:47,534 - __main__ - WARNING - Run feature transformation: ['add', 'exp', 'div', 'minus', 'decisionTreeRegressorTransform']
2024-04-08 15:55:50,038 - __main__ - WARNING - Run feature transformation: ['add', 'exp', 'div', 'negative', 'relu']
2024-04-08 15:55:54,852 - __main__ - WARNING - Run feature transformation: ['add', 'exp', 'div', 'abs', 'xgbRegressorTransform']
2024-04-08 15:56:00,066 - __main__ - WARNING - Add the best feature in set to the best features record.
2024-04-08 15:56:00,067 - __main__ - WARNIN

['add', 'reciprocal', 'negative', 'abs', 'linearRegressorTransform', 'mlpRegressorTransform', 'radian', 'prod', 'timeagg', 'square', 'minus', 'decisionTreeRegressorTransform', 'degree', 'adde', 'cos', 'tanh', 'exp', 'div', 'relu', 'sigmoid', 'kernelapproxrbf', 'xgbRegressorTransform', 'diff', 'ln', 'svrTransform', 'zscore', 'parameter']
W: [1, 1, 3.0, 4, 1, 1, 1, 9.0, 3.0, 3.0, 3.0, 1, 1, 1, 4, 1, 1, 4.0, 2.0, 1, 1, 1, 2.0, 2.0, 3.0, 1]
N: [4, 3, 4, 4, 3, 3, 3, 7, 4, 4, 4, 3, 3, 3, 4, 3, 3, 5, 4, 3, 3, 3, 4, 4, 5, 4]
L: [5.0, 4.0, 4.0, 6.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 7.0, 8.0, 8.0, 4.0, 4.0, 9.0, 9.0, 10, 6.0, 8.0, 6.0, 5.0]
W_rate: [0.16666666666666666, 0.2, 0.42857142857142855, 0.4, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.75, 0.375, 0.375, 0.375, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.36363636363636365, 0.1111111111111111, 0.1111111111111111, 0.5, 0.3333333333333333, 0.1, 0.1, 0.09090909090909091, 0.25, 0.2, 0.

TypeError: unsupported operand type(s) for +: 'MC_node' and 'str'

In [16]:
print(fe.record_top_five)

[('add, exp, div, abs, xgbRegressorTransform', 0.6236388283633548), ('kernelapproxrbf, abs, add, xgbRegressorTransform, mlpRegressorTransform', 0.621259690344617), ('svrTransform, abs, reciprocal, diff, tanh', 0.6172619400368272), ('square, ln, square, div', 0.6172221401652819), ('tanh, cos, radian, tanh, minus', 0.6161494644684237)]


Hier sieht man die Top 5 Transformationen mit dem das Neuronale Netz trainiert wurde.

In [33]:
node = MC_node(State(fe.record_top_five[2][0].split(', ')))

In [34]:
X_train.columns = [str(i) for i in X_train.columns]
X_test.columns = [str(i) for i in X_test.columns]

In [35]:
fe.get_reward_cv_on_test_set2(node)

2024-04-08 16:08:56,156 - __main__ - WARNING - Run feature transformation: ['svrTransform', 'abs', 'reciprocal', 'diff', 'tanh']


(0.5542836102664116,
 ('svr_pred_recip_tanh', 1010    0.196567
  1242    0.170342
  991     0.200003
  471     0.433803
  625     0.325821
            ...   
  715     0.281775
  905     0.218603
  1096    0.183644
  235     0.749011
  1061    0.188392
  Name: svr_pred_recip_tanh, Length: 909, dtype: float32))

In [36]:
X_train2, X_test2, y_train2, y_test2 = fe.pipeline.run_for_dataset(node.get_state().state, X_train, X_test, y_train, y_test)

2024-04-08 16:08:58,594 - __main__ - WARNING - Run feature transformation: ['svrTransform', 'abs', 'reciprocal', 'diff', 'tanh']


In [37]:
model = RandomForestClassifier()
model.fit(X_train2.values, y_train2.values)

print(classification_report(y_test2.values, model.predict(X_test2.values)))


              precision    recall  f1-score   support

           1       0.70      0.51      0.59        51
           2       0.88      0.85      0.87        61
           3       0.99      0.95      0.97       122
           4       0.85      0.96      0.90        23
           5       0.85      0.85      0.85        13
           6       0.75      0.74      0.75       120
           7       0.74      0.82      0.78       193

    accuracy                           0.81       583
   macro avg       0.82      0.81      0.81       583
weighted avg       0.81      0.81      0.81       583

